# mplchart stylesheet (prototype)

In [6]:
from mplchart import styling

In [7]:
stylesheet = styling.get_stylesheet("standard")
stylesheet

In [8]:
stylesheet.config.sections()

['color', 'width', 'linestyle']

In [9]:
print(stylesheet.dumps())

[color]
facecolor = white
ohlc.up = black
ohlc.dn = red
candles.up = black
candles.dn = black
candles.off = white
volume.up = green
volume.dn = red
volume.ma = grey
marker.entry = green
marker.exit = red
tradespan = green
macd = black
macdsignal = blue
macdhist = blue
ppo = black
pposignal = blue
ppohist = blue

[width]
ohlc = 0.8
volume = 0.8
candles = 0.8
macdhist = 0.8
ppohist = 0.8

[linestyle]
line = -


In [10]:
print(stylesheet.get_settings("macd", color="?", alpha=1.0, linestyle="-"))
print(stylesheet.get_settings("macd", color="?"))
print(stylesheet.get_settings("macdsignal", color="?"))
print(stylesheet.get_settings("macdhist", color="?"))

{'color': 'black', 'alpha': 1.0, 'linestyle': '-'}
{'color': 'black'}
{'color': 'blue'}
{'color': 'blue'}
